In [ ]:
import pandas as pd
import wikipedia
import urllib.parse


url = 'https://docs.google.com/spreadsheets/d/1_nMf9q8Hx3t8Q3AnEsqrMkRoPZNnhW57BveGQanCXT0/export?format=csv'
df = pd.read_csv(url)
#print(df)


urls = df['link'].to_list()

def get_last_segment(url):
    # Usa il metodo rsplit per suddividere l'URL dall'ultimo '/'
    segments = url.rsplit('/', 1)
    # Estrai l'ultimo segmento
    last_segment = segments[-1] if segments else ''
    # Decodifica l'URL percent-encoded
    last_segment = urllib.parse.unquote(last_segment)
    # Rimuovi le parentesi tonde
    last_segment = last_segment.replace('(', '').replace(')', '')
    return last_segment




def segmenta_contenuto(content):
    capitoletti = {}
    lines = content.split('\n')
    current_capitolo = ''
    current_sottocapitolo = ''
    current_testo = ''

    for line in lines:
        # Gestisce i capitoli principali (==)
        if line.startswith('== ') and line.endswith(' =='):
            # Salva il contenuto precedente se esiste
            if current_sottocapitolo:
                if current_capitolo not in capitoletti:
                    capitoletti[current_capitolo] = {}
                capitoletti[current_capitolo][current_sottocapitolo] = current_testo.strip()
                current_sottocapitolo = ''
            elif current_capitolo:
                if isinstance(capitoletti.get(current_capitolo), str):
                    # Se è già una stringa, mantienila
                    pass
                else:
                    capitoletti[current_capitolo] = current_testo.strip()

            # Imposta il nuovo capitolo
            current_capitolo = line[3:-3].strip()
            current_testo = ''
            capitoletti[current_capitolo] = {}

        # Gestisce i sottocapitoli (===)
        elif line.startswith('=== ') and line.endswith(' ==='):
            # Salva il contenuto precedente del sottocapitolo se esiste
            if current_sottocapitolo:
                if current_capitolo not in capitoletti:
                    capitoletti[current_capitolo] = {}
                capitoletti[current_capitolo][current_sottocapitolo] = current_testo.strip()

            # Imposta il nuovo sottocapitolo
            current_sottocapitolo = line[4:-4].strip()
            current_testo = ''

        # Aggiunge il testo alla sezione corrente
        else:
            current_testo += line + '\n'

    # Salva l'ultimo contenuto
    if current_sottocapitolo:
        if current_capitolo not in capitoletti:
            capitoletti[current_capitolo] = {}
        capitoletti[current_capitolo][current_sottocapitolo] = current_testo.strip()
    elif current_capitolo:
        if isinstance(capitoletti.get(current_capitolo), dict) and not capitoletti[current_capitolo]:
            capitoletti[current_capitolo] = current_testo.strip()
        elif not isinstance(capitoletti.get(current_capitolo), dict):
            capitoletti[current_capitolo] = current_testo.strip()

    return capitoletti


def get_wikipedia_page_details(page_title):
	page = wikipedia.page(page_title)
	if page:
		content_chapters = segmenta_contenuto(page.content)
		return {
			'summary': page.summary,
			'title': page.title,
			'url': page.url,
			'content': content_chapters,#page.content,  # Contenuto completo
			'links': list(page.links)  # Tutti i link
		}
	else:
		return {
			'summary': page_summary,
			'title': page_title,
			'url': None,
			'content': None,
			'links': None
		}





wikipedia.set_lang("it")
data = []
for x in urls:
	a = get_last_segment(x)
	page_details = get_wikipedia_page_details(a)

	data.append(page_details)


new = pd.DataFrame(data)
new.to_csv("/run/media/raffaele/volumni/sichelia/capa_marianna/extracted_data_wiki/wiki_naples.tsv", sep='\t', index=False)
#print(new['content'][0])
#abba = new['content'][0]
#babba = new['content'][1]

#print(abba.keys())
#print(babba.keys())
#print(new['links'][0])


#capitoletti = segmenta_contenuto(new['content'])
#print(capitoletti)